### This notebook is to provide an example of the Teradata SQL notebook using the Script Table Operator %runsto magic command. It will download and run a Python script on  Teradata Vantage. 

<b>Note that before running the Script Table Operator commands, you want to make sure that Teradata Vantage has Python libraries installed and that the user has the necessary permissions to install/replace/remove files and to run scripts.</b>

<b>The example script uses the `numpy` Python module. Verify that the `numpy` library is installed by running the `%pyinfo module=NUM` magic command.</b>
<p><b>***NOTE: To execute the SQL or magic command in the cell, press SHIFT+ENTER or the run button in the toolbar.</b>

***

### If not already granted, run the necessary GRANT statements for the STO user.
<b>Note that you must connect as a user that is authorized to grant privileges for the STO user.</b>

Replace the STODemoUserName, systemName, user, and host values for your system.

In [ ]:
%var STODemoUserName=<stodemo username>, systemName=<connection name>, user=<user name>, host=<host name or IP>

In [ ]:
GRANT EXECUTE PROCEDURE ON SYSUIF.INSTALL_FILE TO ${STODemoUserName};

In [ ]:
GRANT EXECUTE PROCEDURE ON SYSUIF.REPLACE_FILE TO ${STODemoUserName};

In [ ]:
GRANT EXECUTE PROCEDURE ON SYSUIF.REMOVE_FILE TO ${STODemoUserName};

In [ ]:
GRANT EXECUTE FUNCTION  ON TD_SYSFNLIB.SCRIPT to ${STODemoUserName};

In [ ]:
GRANT EXECUTE ON SYSUIF.DEFAULT_AUTH to ${STODemoUserName};

In [ ]:
%lsconnect

### Add a connection definition for Teradata Vantage if it doesn't exist

In [ ]:
%addconnect name=${systemName}, user=${user}, host=${host}

### Connect 

In [ ]:
%connect ${systemName}

### Show information about the version of Python installed on the active connection

In [ ]:
%pyinfo

### Show information about the version of Python with only the modues starting with "num"

In [ ]:
%pyinfo module=NUM

## Show the help for the runsto magic

In [ ]:
%help runsto

### The cell magic variant of runsto will install the script then run it. The script follows the command line.

### First, make sure the test file is not installed (using the %rmfile magic).

In [ ]:
%rmfile regname=stoTest, prompt=true

### The example script executes a SELECT statement against the `ex2tbl` table, looking for `ObjsID < 400`. You can create a similar table and insert test data or choose a different table and query for your Script Table Operator. 
***
<pre>
CREATE MULTISET TABLE stodemo.ex2tbl ,NO FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP1
     (
      ObsID INTEGER,
      RandSeed FLOAT)
UNIQUE PRIMARY INDEX ( ObsID );

INSERT INTO "stodemo"."ex2tbl"  ("ObsID", "RandSeed")  VALUES("ObsID" [INTEGER ],"RandSeed" [FLOAT ]) ;
***
### Run the following %%runsto magic to install a Python script onto Teradata Vantage.

In [ ]:
%%runsto consumesql = "SELECT COUNT(*) AS nSims, 
       AVG(CAST (oc1 AS INT)) AS AvgCustomers, 
       AVG(CAST (oc2 AS INT)) AS AvgReneged,
       AVG(CAST (oc3 AS FLOAT)) AS AvgWaitTime FROM", \
on=select * from ex2tbl where ObsID < 400,\
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",\
filename=stoTest.py, regname=stoTest, replacefile=false

"""
* The contents of this file are Teradata Public Content 
* and have been released to the Public Domain.
* Please see license.txt file in the package for more information.
* Alexander Kolovos - 4 Sep 2015 - v.1.2
* Copyright 2019-2020 Teradata. All rights reserved.
* Licensed under BSD

R And Python Analytics with the SCRIPT Table Operator
Orange Book supplementary material
Example 2 - Simulation - Python

Simulation Bank renege example taken from:
https://simpy.readthedocs.org/en/latest/examples/bank_renege.html
Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

Requires random, numpy, sys and simpy packages.

Required input:
- table ex2tbl defined in ex2dataTblDef.txt
- ex2tbl data from file ex2data.txt

Input parameters:
INTERVAL_CUSTOMERS : Time interval (minutes) between customer entries
MIN_PATIENCE       : Minimum time (minutes) customers will wait
MAX_PATIENCE       : Maximum time (minutes) customers will wait
TIME_IN_BANK       : Time (minutes) a customer spends being served
MAX_MINUTES        : Process observation time (minutes)

Output:
Number of customers that visited in MAX_MINUTES
Number of customers that reneged in MAX_MINUTES
Average waiting time per customer

"""
import random
import simpy
import numpy as np
import sys

DELIMITER='\t'

# Load data from table, one row at a time
# Expecting table with 2 columns: ObsID, RandomSeed
tbldatalist = []
while 1:
    try:
        line = raw_input()
        if line == '':   # Exit if user provides blank line
            break
        else:
            allnum = line.split(DELIMITER)
            allnum = [x.replace(" ","") for x in allnum]
            tbldatalist.append(allnum)
    except (EOFError):   # Exit if reached EOF or CTRL-D
        break
        
ndl = len(tbldatalist)

# If the present AMP has no data, then exit this script instance.
if ndl == 0:
    sys.exit()

# Assumes data set has ObsID in col 1, RandomSeed in col 2
nDataCols = 2
# Convert list into array
tbldata = np.array(tbldatalist).reshape(ndl,nDataCols)
del tbldatalist

obsID = tbldata[:,0]
randSeed = tbldata[:,1]

# set the variables to default values in case they're not specified on the command line
INTERVAL_CUSTOMERS = 4
MIN_PATIENCE = 5
MAX_PATIENCE = 10
TIME_IN_BANK = 6
MAX_MINUTES = 480

if len(sys.argv) > 1:
    INTERVAL_CUSTOMERS = sys.argv[1]
if len(sys.argv) > 2:
    MIN_PATIENCE = sys.argv[2]
if len(sys.argv) > 3:
    MAX_PATIENCE = sys.argv[3]
if len(sys.argv) > 4:
    TIME_IN_BANK = sys.argv[4]
if len(sys.argv) > 5:
    MAX_MINUTES = sys.argv[5]

RANDOM_SEED = 20           # A default random seed value. Here, seeds are input
INTERVAL_CUSTOMERS = 4.0   # Generate new customers roughly every x minutes
MIN_PATIENCE = 5           # Min. customer patience
MAX_PATIENCE = 10          # Max. customer patience
TIME_IN_BANK = 6.0         # Minutes a customer spends being served
MAX_MINUTES = 480          # Minutes to observe system behavior

def source(env, interval, counter):
    """Source generates customers randomly"""
    global iCust
    iCust = 0
    while 1:
        iCust += 1
        c = customer(env, 'Customer%02d' % iCust, counter, TIME_IN_BANK)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive
        global totWait
        totWait += wait
        global totReneged

        if req in results:
            # We got to the counter

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)

        else:
            # We reneged
            totReneged += 1

# Setup and start the simulation
for i in range( 0, len(randSeed) ):
    global totWait
    totWait = 0
    global totReneged
    totReneged = 0
    seedNo = randSeed[i]
    random.seed( seedNo )
    env = simpy.Environment()
    
    # Start processes and run
    counter = simpy.Resource(env, capacity=1)
    env.process(source(env, INTERVAL_CUSTOMERS, counter))
    env.run( until=MAX_MINUTES )
    
    print long(iCust), DELIMITER, long(totReneged), DELIMITER, totWait/iCust


In [ ]:
%showfile stoTest

### Once the file has been installed, it can be run using the %runsto magic without including the script.
#### Run the SCRIPT table operator with only required parameters (script file already installed)
* ON specifies the contents of a table as the input to the script
* RETURNS specifies the output columns
* FILENAME specifies the script file name

All the output from the script is returned (no "consumesql" parameter)
#### In the example below we are referring to an existing table called `ex2tblshort` to provide input to the script. You can create the similar table and load data into it or use one on your own tables.
***
<pre>
CREATE MULTISET TABLE stodemo.ex2tblshort ,NO FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP1
     (
      ObsID INTEGER,
      RandSeed FLOAT)
UNIQUE PRIMARY INDEX ( ObsID );

INSERT INTO "stodemo"."ex2tblshort"  ("ObsID", "RandSeed")  VALUES("ObsID" [INTEGER ],"RandSeed" [FLOAT ]) ;
***

In [ ]:
%runsto 
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

### Run the SCRIPT table operator as above but with the COMMAND parameter.
#### The "command" parameter is specified because the script is being given command-line arguments. The "%s" indicates where the script file name is to be inserted into the conmand.

In [ ]:
%runsto 
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py,
command= "python %s 4 5 10 6 480"

### Run the SCRIPT table operator as before except count and average the script output columns
#### The "consumesql" parameter specifies a query that will read the script's output data

In [ ]:
%runsto 
consumesql = "SELECT COUNT(*) AS nSims,
       AVG(CAST (oc1 AS INT)) AS AvgCustomers, 
       AVG(CAST (oc2 AS INT)) AS AvgReneged,
       AVG(CAST (oc3 AS FLOAT)) AS AvgWaitTime FROM",
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

### Run the SCRIPT table operator with the results of a query as its input
#### The "on" parameter specifies a query

In [ ]:
%runsto 
consumesql = "SELECT COUNT(*) AS nSims,
       AVG(CAST (oc1 AS INT)) AS AvgCustomers, 
       AVG(CAST (oc2 AS INT)) AS AvgReneged,
       AVG(CAST (oc3 AS FLOAT)) AS AvgWaitTime FROM",
on=select * from ex2tbl where ObsID < 400,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

### Run the SCRIPT table operator and insert its output into another table.
#### The "consumesql" parameter specifies an INSERT/SELECT that inserts the results of the script into a table.

In [ ]:
drop table stodemo.STAGE;

In [ ]:
CREATE TABLE stodemo.STAGE
     (
      c1 varchar(10),
      c2 varchar(10),
      c3 varchar(50));

In [ ]:
%runsto
consumesql = "INSERT INTO stodemo.STAGE(c1, c2, c3) SELECT * FROM",
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

In [ ]:
SELECT * FROM stodemo.STAGE;

### The following two cells illustrate what is being run when the %runsto magic is run

In [ ]:
SET SESSION SEARCHUIFDBPATH = "STODEMO"

In [ ]:
SELECT * FROM 
SCRIPT(
      ON ex2tblshort
      SCRIPT_COMMAND('python ./STODEMO/stoTest.py')
      RETURNS ('oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)')
)

### Run a script that contains a syntax error. The lines in the script log from the server that are the result of running the script are shown.
#### The example below assumes there is an `ex2pERR.py` file on the system. You can use the above example `stoTest.py` and force an error in the script to demonstrate the error handling.

In [ ]:
%runsto 
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=ex2pERR.py

In [ ]:
%showfile ex2pERR

### Script files (and data files) can be installed without running them using the %installfile magic

In [ ]:
%help installfile

#### Run the %rmfile to remove the file in case it is already installed.

In [ ]:
%rmfile regname=stoTest2

#### Install the file
The localfile parameter value can include the full path name of the file or it can be relative to the current directory.
If the file is a binary file, you will need to set the text parameter value to false.
#### The example below assumes there is a Python file called `ex2p.py` on your local machine.

In [ ]:
%installfile regname=stoTest2, filename=stoTest2.py, localfile=ex2p.py, text=true

#### If the file gets modified and needs to be replaced, include the "replace=true" parameter.
The text parameter can be omitted for text files.

In [ ]:
%installfile regname=stoTest2, filename=stoTest2.py, localfile=ex2p.py, replace=true

#### Remove the file.
The prompt=false parameter says that confirmation isn't required'

In [ ]:
%rmfile regname=stoTest2, prompt=false

Copyright 2019-2020 Teradata. All rights reserved.